### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221118_scenario_projections.ipynb")

2022/11/17 15:00:06 INFO|============================================================


### Fit an initial scenario through February 2022

In [ ]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

### Create and run scenarios from Feb 2022 to present

In [10]:
multiprocess = 5

scenario_params = json.load(open('covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json'))

model_args = {
    'base_spec_id': 4386, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for (weak_escape, strong_escape) in [(0.6, 0.15), (0.66, 0.165), (0.72, 0.18),
                                     (0.54, 0.135), (0.48, 0.12)]:
    emv_seed = [{"param": "emv_seed", "attrs": None, "vals": {"2020-01-01": 0, "2022-09-15": 10,
                                                                       "2022-10-30": 0}}]
    hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
           "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
    weak_param = [{"param": "immune_escape",
                   "from_attrs": {"immun": "weak",
                                  "variant": ["none", "wildtype", "alpha", "delta",
                                              "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["emv"]},
                   "vals": {"2020-01-01": weak_escape},
                   "desc": "Scrabble variants immune escape value, weak immunity"}]
    strong_param = [{"param": "immune_escape",
                   "from_attrs": {"immun": "strong",
                                  "variant": ["none", "wildtype", "alpha", "delta",
                                              "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["emv"]},
                   "vals": {"2020-01-01": strong_escape},
                   "desc": "Scrabble variants immune escape value, strong immunity"}]
    lt5_vacc_adjust = [{"param": "immunity",
                        "attrs": {'age': '0-19', 'vacc': 'shot1'},
                        "mults": {"2020-01-01": 1,
                                  "2022-06-24": 0.99 + 0.01*vacc_eff_lt5,
                                  "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                  "2022-07-08": 0.97 + 0.03*vacc_eff_lt5,
                                  "2022-07-19": 0.96 + 0.04*vacc_eff_lt5,
                                  "2022-07-29": 0.95 + 0.05*vacc_eff_lt5,
                                  "2022-08-11": 0.94 + 0.06*vacc_eff_lt5,
                                  "2022-08-30": 0.93 + 0.07*vacc_eff_lt5,
                                  "2022-09-26": 0.92 + 0.08*vacc_eff_lt5,
                                  "2022-10-26": 0.91 + 0.09*vacc_eff_lt5,},
                        "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
    scenario_model_args.append({'params_defs': scenario_params + weak_param + strong_param + emv_seed + lt5_vacc_adjust,
                                'hosp_reporting_frac': hrf,
                                'tags': {'emv_escape_weak': weak_escape,
                                        'emv_escape_strong': strong_escape}})
            

In [11]:
# check how many scenarios there are
len(scenario_model_args)

5

In [12]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/11/17 15:23:22 INFO|============================================================
2022/11/17 15:23:22 INFO|============================================================
2022/11/17 15:23:22 INFO|============================================================
2022/11/17 15:23:22 INFO|============================================================
2022/11/17 15:23:22 INFO|============================================================
2022/11/17 15:23:25 INFO|-----------|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.15} Retrieving vaccinations data
2022/11/17 15:23:25 INFO|-----------|{'emv_escape_weak': 0.48, 'emv_escape_strong': 0.12} Retrieving vaccinations data
2022/11/17 15:23:25 INFO|-----------|{'emv_escape_weak': 0.72, 'emv_escape_strong': 0.18} Retrieving vaccinations data
2022/11/17 15:23:25 INFO|-----------|{'emv_escape_weak': 0.54, 'emv_escape_strong': 0.135} Retrieving vaccinations data
2022/11/17 15:23:25 INFO|-----------|{'emv_escape_weak': 0.66, 'emv_escape_strong': 0.165} Ret

2022/11/17 15:25:45 INFO|--------|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.15, 'fit_batch': '1'}: Transmission control fit 2/7 completed in 15.291243415999986 seconds: {795: {'co': 0.8121421746885041}, 809: {'co': 0.8306571755757243}, 823: {'co': 0.8365104710949993}, 837: {'co': 0.8398687683987986}, 851: {'co': 0.826853485154081}, 865: {'co': 0.8509032621694365}}
2022/11/17 15:25:45 INFO|---------|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.15, 'fit_batch': '1'}: Running forward sim
2022/11/17 15:25:46 INFO|--------|{'emv_escape_weak': 0.72, 'emv_escape_strong': 0.18, 'fit_batch': '1'}: Transmission control fit 2/7 completed in 15.631082875000004 seconds: {795: {'co': 0.8121421746885041}, 809: {'co': 0.8306571755757243}, 823: {'co': 0.8365104710949993}, 837: {'co': 0.8398687683987986}, 851: {'co': 0.826853485154081}, 865: {'co': 0.8509032621694365}}
2022/11/17 15:25:46 INFO|---------|{'emv_escape_weak': 0.72, 'emv_escape_strong': 0.18, 'fit_batch': '1'}: Running forward sim


2022/11/17 15:26:33 INFO|--------|{'emv_escape_weak': 0.54, 'emv_escape_strong': 0.135, 'fit_batch': '4'}: Transmission control fit 5/7 completed in 13.135833125000005 seconds: {879: {'co': 0.8389511485832544}, 893: {'co': 0.8336263597956938}, 907: {'co': 0.8361920361828573}, 921: {'co': 0.8393465731914771}, 935: {'co': 0.8154959008837471}, 949: {'co': 0.8312433074420906}}
2022/11/17 15:26:33 INFO|---------|{'emv_escape_weak': 0.54, 'emv_escape_strong': 0.135, 'fit_batch': '4'}: Running forward sim
2022/11/17 15:26:34 INFO|--------|{'emv_escape_weak': 0.48, 'emv_escape_strong': 0.12, 'fit_batch': '4'}: Transmission control fit 5/7 completed in 13.193673749999988 seconds: {879: {'co': 0.8389511485832544}, 893: {'co': 0.8336263597956938}, 907: {'co': 0.8361920361828573}, 921: {'co': 0.8393465731914771}, 935: {'co': 0.8154959008837471}, 949: {'co': 0.8312433074420906}}
2022/11/17 15:26:34 INFO|---------|{'emv_escape_weak': 0.48, 'emv_escape_strong': 0.12, 'fit_batch': '4'}: Running forwar

2022/11/17 15:27:06 INFO|--------|{'emv_escape_weak': 0.72, 'emv_escape_strong': 0.18, 'fit_batch': '6'}: Transmission control fit 7/7 completed in 15.325036249999982 seconds: {935: {'co': 0.8152013773459715}, 949: {'co': 0.8322615986067066}, 963: {'co': 0.8011137501244704}, 977: {'co': 0.8185382646889103}, 991: {'co': 0.7756889028378703}, 1005: {'co': 0.8059063691902103}}
2022/11/17 15:27:06 INFO|---------|{'emv_escape_weak': 0.72, 'emv_escape_strong': 0.18, 'fit_batch': '6'}: Running forward sim
2022/11/17 15:27:06 INFO|--------|{'emv_escape_weak': 0.48, 'emv_escape_strong': 0.12, 'fit_batch': '6'}: Transmission control fit 7/7 completed in 13.088201124999983 seconds: {935: {'co': 0.8151297825781736}, 949: {'co': 0.8314885919035341}, 963: {'co': 0.8029849263734442}, 977: {'co': 0.8118647104569519}, 991: {'co': 0.7804172231904645}, 1005: {'co': 0.7404154383132662}}
2022/11/17 15:27:06 INFO|---------|{'emv_escape_weak': 0.48, 'emv_escape_strong': 0.12, 'fit_batch': '6'}: Running forwar

2022/11/17 15:27:52 INFO|-----------------------|spec_ids: 4404,4405,4406,4407,4408


### Run the report for each fit model

In [13]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45', 'emv'], from_date='2022-01-01')

2022/11/17 15:35:52 INFO|============================================================
2022/11/17 15:35:52 INFO|--------|Solving model
2022/11/17 15:35:53 INFO|============================================================
2022/11/17 15:35:53 INFO|--------|Solving model
2022/11/17 15:35:54 INFO|============================================================
2022/11/17 15:35:54 INFO|--------|Solving model
2022/11/17 15:35:55 INFO|============================================================
2022/11/17 15:35:55 INFO|--------|Solving model
2022/11/17 15:35:56 INFO|============================================================
2022/11/17 15:35:56 INFO|--------|Solving model
2022/11/17 15:35:57 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/17 15:35:58 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored wh

In [14]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/11/17 15:38:21 INFO|Projecting
2022/11/17 15:38:21 INFO|
2022/11/17 15:38:43 INFO|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.15, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/17 15:38:45 INFO|
2022/11/17 15:39:09 INFO|{'emv_escape_weak': 0.66, 'emv_escape_strong': 0.165, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/17 15:39:11 INFO|
2022/11/17 15:39:32 INFO|{'emv_escape_weak': 0.72, 'emv_escape_strong': 0.18, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/17 15:39:34 INFO|
2022/11/17 15:39:57 INFO|{'emv_escape_weak': 0.54, 'emv_escape_strong': 0.135, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/17 15:39:58 INFO|
2022/11/17 15:40:21 INFO|{'emv_escape_weak': 0.48, 'emv_escape_strong': 0.12, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/17 15:40:23 INFO|Running reports


In [ ]:
model.solution_ydf.to_csv('covid_model/output/20221109_scenario_projections.ipynb/state_model_solution_ydf.csv', index=False)